# Challenge: Promotions

In this challenge, you'll develop codes to parse and analyze data returned from another API on Zalando such as [Promos homme (Men's Promotions)
](https://www.zalando.fr/promo-homme/) or [Promos femme (Women's Promotions)](https://www.zalando.fr/promo-femme/). The workflow is almost the same as in the guided lesson but you'll work with different data.

## Obtaining the link

Wrote your codes in the cell below to obtain the data from the API endpoint you choose. A recap of the workflow:

1. Examine the webpages and choose one that you want to work with.

1. Use Google Chrome's DevTools to inspect the XHR network requests. Find out the API endpoint that serves data to the webpage.

1. Test the API endpoint in the browser to verify its data.

1. Change the page number offset of the API URL to test if it's working.

In [1]:
# your code here
url='https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset=84&sort=popularity'

## Reading the data

In the next cell, use Python to obtain data from the API endpoint you chose in the previous step. Workflow:

1. Import libraries.

1. Define the initial API endpoint URL.

1. Make request to obtain data of the 1st page. Flatten the data and store it in an empty object variable.

1. Find out the total page count in the 1st page data.

1. Use a FOR loop to make requests for the additional pages from 2 to page count. Append the data of each additional page to the flatterned data object.

1. Print and review the data you obtained.

In [2]:
# your code here
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
response = requests.get(url, headers=headers).json()

In [4]:
normalized_data = json_normalize(response)
normalized_data_articles = json_normalize(normalized_data["articles"][0])
normalized_data_articles.head()

,sku,name,sizes,url_key,media,brand_name,is_premium,family_articles,flags,product_group,delivery_promises,price.original,price.promotional,price.has_different_prices,price.has_different_original_prices,price.has_different_promotional_prices,price.has_discount_on_selected_sizes_only,outfits
0,TI112D04U-Q11,KILLINGTON - Bottines à lacets - black,"[40, 41, 41.5, 42, 43, 43.5, 44, 44.5, 45, 45....",timberland-killington-bottines-a-lacets-black-...,[{'path': 'TI/11/2D/04/UQ/11/TI112D04U-Q11@11....,Timberland,False,"[{'sku': 'TI112D04U-Q11', 'url_key': 'timberla...","[{'key': 'discountRate', 'value': '-20%', 'tra...",shoe,[],"159,95 €","127,95 €",False,False,False,False,NaN
1,TI112M00F-Q11,CITYROAM CHUKKA - Baskets montantes - black co...,"[40, 43, 44, 44.5, 45, 45.5, 46]",timberland-cityroam-cupsole-chukka-baskets-mon...,[{'path': 'TI/11/2M/00/FQ/11/TI112M00F-Q11@12....,Timberland,False,"[{'sku': 'TI112M00F-Q11', 'url_key': 'timberla...","[{'key': 'discountRate', 'value': '-20%', 'tra...",shoe,[],"149,95 €","119,95 €",False,False,False,False,NaN
2,L0642D003-K13,HERREN KURZARM - Polo - marine,"[S, M, L, XL, XXL, 3XL]",lacoste-sport-polo-dark-blue-l0642d003-k13,[{'path': 'L0/64/2D/00/3K/13/L0642D003-K13@17....,Lacoste Sport,False,"[{'sku': 'L0642D003-K13', 'url_key': 'lacoste-...","[{'key': 'discountRate', 'value': '-15%', 'tra...",clothing,[],"69,95 €","59,45 €",False,False,False,False,NaN
3,PI912K05M-Q11,Bottines à lacets - black,"[40, 41, 43, 44, 46]",pier-one-bottines-a-lacets-black-pi912k05m-q11,[{'path': 'PI/91/2K/05/MQ/11/PI912K05M-Q11@10....,Pier One,False,"[{'sku': 'PI912K05M-Q11', 'url_key': 'pier-one...","[{'key': 'discountRate', 'value': '-70%', 'tra...",shoe,[],"69,95 €","20,99 €",False,False,False,False,NaN
4,LA222P00Q-G11,Polo - bordeaux,"[XS, S, M, L, XL, XXL, 3XL, 4XL]",lacoste-slim-fit-polo-la222p00q-g11,[{'path': 'LA/22/2P/00/QG/11/LA222P00Q-G11@10....,Lacoste,False,"[{'sku': 'LA222P00Q-G11', 'url_key': 'lacoste-...","[{'key': 'discountRate', 'value': '-20%', 'tra...",clothing,[],"94,95 €","75,95 €",False,False,False,False,NaN


In [5]:
total_pages=response['pagination']['page_count']
total_pages

593

In [ ]:
dataset = pd.DataFrame()
for i in range(total_pages):
    url = f"https://www.zalando.fr/api/catalog/articles?categories=promo-homme&limit=84&offset={84*i}&sort=popularity"
    json = requests.get(url, headers=headers).json()
    normalized_data = json_normalize(json)
    normalized_data_articles = json_normalize(normalized_data["articles"][0])
    dataset = dataset.append(normalized_data_articles, sort=False).copy()
dataset.head()

## Bonus

Extract the following information from the data:

* The trending brand.

* The product(s) with the highest discount.

* The sum of discounts of all goods (sum_discounted_prices divided by sum_original_prices).

In [ ]:
# your code here